In [20]:
import pandas as pd
import pandas_ml as pdml
import numpy as np
from sklearn import cross_validation, metrics   #Additional scklearn functions

from sklearn.feature_extraction import DictVectorizer as DV
from sklearn.metrics import roc_auc_score as AUC

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

pd.set_option('display.max_rows', 8)
pd.set_option('display.max_rows', 6)

train_df = pd.read_csv('data3.csv',header=0, index_col=0)
test_df = pd.read_csv('quiz.csv',header=0, index_col=0)
print(train_df.shape)
# convert to pdml.ModelFrame
#train_df = pdml.ModelFrame(train_df,target='label')
train_df=train_df.dropna()
#test_df = pdml.ModelFrame(test_df)
test_df=test_df.dropna()

print(test_df.shape)

(126837, 52)
(31709, 51)


In [21]:
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)

feature_columns_to_use = ([x for x in train_df.columns])
feature_columns_to_use.pop(-1)

numeric_cols = ['59','60']
nonnumeric_columns = feature_columns_to_use[0:46]+feature_columns_to_use[48:]

# Join the features from train and test together before imputing missing values,
# in case their distribution is slightly different
big_X = train_df[feature_columns_to_use].append(test_df[feature_columns_to_use])
big_X_imputed = DataFrameImputer().fit_transform(big_X)
#print(big_X_imputed)

In [22]:
# To handle categorical features, we need to change
# them to columns of integer values.
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for feature in nonnumeric_columns:
    big_X_imputed[feature] = le.fit_transform(big_X_imputed[feature])

# Prepare the inputs for the model
train_X = big_X_imputed[0:train_df.shape[0]].as_matrix()
test_X = big_X_imputed[train_df.shape[0]::].as_matrix()
train_y = train_df['label']

In [27]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# Create and fit an AdaBoosted decision tree
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=40, criterion='gini'),
                         algorithm="SAMME.R", 
                         n_estimators=2000)

bdt.fit(train_X, train_y)
predictions = bdt.predict(test_X)

In [28]:
indx=[x for x in range(1,31710)]
submission = pd.DataFrame({'Id': indx,
                           'Prediction': predictions })
submission.to_csv("submission.csv", index=False, sep=",")